## Discretisations


- Underworld's model building blocks

**New concepts**: FEM Mesh, 'data', MeshVariables, 'specialSets'

<table><tr><td><img src='../images/refinement.png'></td><td><img src='../images/gradient.png'></td></tr></table>

In [ ]:
# we import our python modules
import underworld as uw
import glucifer

In [ ]:
# lets make a mesh and visualise it
mesh = uw.mesh.FeMesh_Cartesian(elementType = 'Q1', 
                                elementRes  = (8,8),
                                minCoord    = (0.0,0.0),
                                maxCoord    = (10.,10.0))

# to visualise - common pattern
fig = glucifer.Figure(rulers=True, rulerticks=5, border=0) # define figure
fig.append(glucifer.objects.Mesh(mesh, nodeNumbers=True))  # add object to render
fig.show()                                                 # render now

## the mesh object

In [ ]:
# hold shift + tabx2 with the cursur on the mesh
mesh

In [ ]:
# The meshes' node geometry data can be directly read via numpy arrays `mesh.data` .. try it

In [ ]:
#mesh.data

#### The 'data' is attribute is a numpy array 
- we have direct access to it
- same memory as the underlying c code layer

In [ ]:
# the mesh.data is a read only array, to alter it we need the 'with' pattern
with mesh.deform_mesh():
    mesh.data[0][0] -= 0.5

In [ ]:
# no need to redefine the 'fig' object - it's persistent 
fig.show()

In [ ]:
# a quick example for mesh refinement
mesh.reset()
with mesh.deform_mesh():
    for index, coord in enumerate(mesh.data):
        factor = (mesh.data[index][1]/mesh.maxCoord[1])**0.5
        mesh.data[index][1] = factor*mesh.maxCoord[1]
fig.show()

In [ ]:
# reset to equally spaced
mesh.reset()

### Adding a variable to the mesh: a MeshVariable

In [ ]:
# number of components can be set with 'nodeDofCount'
field = uw.mesh.MeshVariable(mesh=mesh, nodeDofCount=1)

In [ ]:
fig2 = glucifer.Figure()
fig2.append(glucifer.objects.SurfaceOnMesh(mesh, field))
fig2.show()

In [ ]:
# the fields.data is also a numpy array, direct access
field.data[40] = 4.

In [ ]:
fig2.show()

In [ ]:
for index in range(mesh.nodesGlobal):
    field.data[index] = index

In [ ]:
fig2.show()

In [ ]:
# something more interesting, a sinusoidal shape

# note the import
import math

for index, coord in enumerate(mesh.data):
    field.data[index] = math.sin(coord[0]*2*math.pi/10. )#(coord[0]/2.)**2+coord[1]
#     field.data[index] = math.exp( -1.0/10.*((coord[0]-5.0)**2+(coord[1]-5.0)**2))

fig2.show()

In [ ]:
# with MeshVariable you have access to its spatial gradient with `field.fn_gradient`
# the gradient of a scalar field is a vector
fig3 = glucifer.Figure(**fig2)
fig3.append( glucifer.objects.VectorArrows( mesh, field.fn_gradient,     
                                           scaling=1., arrowHead=0.35))

fig3.show()

In [ ]:
# zero the whole field with numpy 
field.data[...] = 0.0

#### mesh.specialSets

In [ ]:
# Sets of node indexes that define boundaries are available under the `mesh.specialSets` dictionary
print mesh.specialSets.keys()
# the right wall is defined as MaxI_VertexSet
rightWall = mesh.specialSets["MaxI_VertexSet"]
print rightWall

In [ ]:
# get the coordinates along `rightWall`
for index in rightWall:
    print mesh.data[index]

In [ ]:
fig.show()

## Points on parallisation
- The `Mesh` is automatically decomposed across job processors.
- The `MeshVariable`s are similarly decomposed with the `Mesh`.
- glucifer can render parallel automatically but only to disk, ie. `show` --> `save_image`
- `Mesh` and `MeshVariable`s can be saved/load to xdmf file format.

**Exercise:**

1. Use specialSets to set each boundary wall of `field` to constant value
2. Can you save the mesh to xdmf, visualise it in paraview, can you reload it?

In [ ]:
field.data[...] = 1.0
for index in mesh.specialSets['MinI_VertexSet']:
    field.data[index] = 1.0
for index in mesh.specialSets['MaxI_VertexSet']:
    field.data[index] = 2.0
for index in mesh.specialSets['MinJ_VertexSet']:
    field.data[index] = 3.0
for index in mesh.specialSets['MaxJ_VertexSet']:
    field.data[index] = 4.0

In [ ]:
fig2.show()

In [ ]:
mH = mesh.save('mymesh.h5')
fH = field.save('myfield.h5', mH)
field.xdmf('myfield', fH, 'temperature', mH, 'mesh')